In [27]:
import requests
import pandas as pd 
import time
import numpy as np
from bs4 import BeautifulSoup

In [2]:
amsterdam = 89
rotterdam = 361
hamburg = 104
ports = ['AMSTERDAM', 'ROTTERDAM', 'HAMBURG']

In [3]:
#TODO: URL automtisieren
origURL = 'https://www.myshiptracking.com/ports-arrivals-departures/?mmsi=&pid=&type=0&time=1575212940_1575299340&pp=50&page='

data = []
i = 1
while(i <= 50):
    url = origURL + '%d' % i

    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    rows = soup.select('div.cs-table div.table-group')
    for row in rows:
        list = row.select_one('div.table-row').text.strip().splitlines()
        type = row.select_one('div.table-row div.td_vesseltype img')
        list.append(type.get('src'))
        data.append(list)
    i+= 1
    time.sleep(0)
    
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['Event', 'Time', 'Port', 'Vessel', 'VesselType']) 

In [4]:
def setVesselType(number):
    type = '';
    if(number == '6_'):
        type = 'passenger';
    if(number == '7_'):
        type = 'cargo';
    if(number == '8_'):
        type = 'tanker';
    if(number == '3_'):
        type = 'tug';
    if(number == '0_'):
        type = 'undefined';
    if(number == '9_'):
        type = 'yacht';
    if(number == '4_'):
        type = 'high speed';
    if(number == '10'):
        type = 'fishing';
    if(number == '13'):
        type = 'navigation aid';
    return type;

In [5]:
for i in df.index:
    number = df['VesselType'][i][11:13]
    df['VesselType'][i] = setVesselType(number)

In [6]:
print(df) 

          Event              Time                     Port  \
0       Arrival  2019-12-02 15:09                     TEES   
1       Arrival  2019-12-02 15:08                ROTTERDAM   
2       Arrival  2019-12-02 15:08                   ROUSAY   
3       Arrival  2019-12-02 15:08          CALETA LA ARENA   
4       Arrival  2019-12-02 15:08                BAKAUHENI   
5     Departure  2019-12-02 15:08                LIVERPOOL   
6     Departure  2019-12-02 15:08                 CRUZ BAY   
7       Arrival  2019-12-02 15:08               SALTHOLMEN   
8       Arrival  2019-12-02 15:08                HONG KONG   
9     Departure  2019-12-02 15:08                 RAMSGATE   
10    Departure  2019-12-02 15:08            VASHON ISLAND   
11    Departure  2019-12-02 15:08                 GELKENES   
12      Arrival  2019-12-02 15:08              SCHOONHOVEN   
13    Departure  2019-12-02 15:08                 CHANGSHU   
14    Departure  2019-12-02 15:08                   WEIHAI   
15    De

In [7]:
df.to_csv("data.csv")

In [57]:
#set of vessels
filtered_df = df[df['Port'].isin(ports)]
vessels = set(df.Vessel)

In [58]:
timetable = pd.DataFrame([], columns = ['DeparturePort', 'DepartureTime', 'ArrivalPort', 'ArrivalTime', 'Vessel', 'VesselType'])

In [59]:
### Create Vessel specific table
for v in vessels:
    allEntriesOfVessel = df[df['Vessel'].isin([v])]
    
    # Set für Vessels erstellen, für jedes Vessel obige Tabelle erstellen
    depFirst = False
    for index, row in allEntriesOfVessel[::-1].iterrows():
        if(allEntriesOfVessel.Event[index] == "Departure"):
            departurePort = (allEntriesOfVessel.Port[index])
            departureTime = (allEntriesOfVessel.Time[index])
            vessel = (allEntriesOfVessel.Vessel[index])
            vesselType = (allEntriesOfVessel.VesselType[index])
            depFirst = True
        if(allEntriesOfVessel.Event[index] == "Arrival" and depFirst):
            arrivalPort = (allEntriesOfVessel.Port[index])
            arrivalTime = (allEntriesOfVessel.Time[index])
            depFirst = False

            newEntry = pd.DataFrame([[departurePort, departureTime, arrivalPort, arrivalTime, vessel, vesselType]], columns = ['DeparturePort', 'DepartureTime', 'ArrivalPort', 'ArrivalTime', 'Vessel', 'VesselType'])
            timetable = timetable.append(newEntry, ignore_index = True)


In [60]:
print(timetable)
timetable.to_csv("timetable.csv")

              DeparturePort     DepartureTime              ArrivalPort  \
0                  BOSTANCI  2019-12-02 14:34                 BOSTANCI   
1                   MONDORF  2019-12-02 14:39                  MONDORF   
2                   MONDORF  2019-12-02 14:47                  MONDORF   
3                   MONDORF  2019-12-02 14:57                  MONDORF   
4                  BLECKEDE  2019-12-02 14:40                 BLECKEDE   
5                  BLECKEDE  2019-12-02 14:54                 BLECKEDE   
6                ZWARTSLUIS  2019-12-02 14:50    MILLINGEN AAN DE RIJN   
7    MAASVLAKTE - ROTTERDAM  2019-12-02 14:37   MAASVLAKTE - ROTTERDAM   
8                  SURALAYA  2019-12-02 14:48                    MERAK   
9                 LEMWERDER  2019-12-02 14:38                LEMWERDER   
10                LEMWERDER  2019-12-02 14:55                LEMWERDER   
11             BOHUS MALMON  2019-12-02 14:32             BOHUS MALMON   
12                   BOSTON  2019-12-0

In [46]:
# Set für Vessels erstellen, für jedes Vessel obige Tabelle erstellen
depFirst = False
for index, row in pont_6[::-1].iterrows():
    if(pont_6.Event[index] == "Departure"):
        departurePort = (pont_6.Port[index])
        departureTime = (pont_6.Time[index])
        vessel = (pont_6.Vessel[index])
        vesselType = (pont_6.VesselType[index])
        depFirst = True
    if(pont_6.Event[index] == "Arrival" and depFirst):
        arrivalPort = (pont_6.Port[index])
        arrivalTime = (pont_6.Time[index])
        depFirst = False
        
        newEntry = pd.DataFrame([[departurePort, departureTime, arrivalPort, arrivalTime, vessel, vesselType]], columns = ['DeparturePort', 'DepartureTime', 'ArrivalPort', 'ArrivalTime', 'Vessel', 'VesselType'])
        timetable = timetable.append(newEntry, ignore_index = True)

In [47]:
print(timetable)

  DeparturePort     DepartureTime ArrivalPort       ArrivalTime       Vessel  \
0       ZAANDAM  2019-12-02 14:33   AMSTERDAM  2019-12-02 14:34  PONT 6 [NL]   
1     AMSTERDAM  2019-12-02 14:43     ZAANDAM  2019-12-02 14:44  PONT 6 [NL]   
2       ZAANDAM  2019-12-02 14:53   AMSTERDAM  2019-12-02 14:53  PONT 6 [NL]   
3     AMSTERDAM  2019-12-02 15:04     ZAANDAM  2019-12-02 15:05  PONT 6 [NL]   

  VesselType  
0  passenger  
1  passenger  
2  passenger  
3  passenger  
